In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np
import numpy.linalg as la
from scipy.integrate import RK45

import sympy as sym
# Heaviside numerics workaround
sympy_modules = [{'Heaviside': lambda x: np.heaviside(x,0.5)}, 'numpy']

from table_maker import *
from functools import partial, reduce
from itertools import *
from math import ceil

import pickle

def cos_bell(x, center=0, width=2*np.pi, height=1):
    return (np.cos((x-center)/width*2*np.pi)+1)/2*height * np.heaviside(x-center+width/2,0) * np.heaviside(-x+center+width/2,0)

import collections.abc
def collect_eqn(expr, terms):
    if not isinstance(terms, collections.abc.Container):
        term = terms # isn't a list or tuple
        left, right = 0,0
        for my_expr in expr.args:
            has_term = False
            for my_term in sym.postorder_traversal(my_expr):
                if term == my_term:
                    has_term = True
                    break
            if has_term:
                left += my_expr
            else:
                right -= my_expr
        return sym.Eq(left, right, evaluate=False)
    else: # isn't a list or tuple
        left_total, right_total = 0, expr
        for term in terms:
            left, right = collect_eqn(right_total, term).args
            left_total += left
            right_total = -right
        return sym.Eq(left_total, right_total, evaluate=False)
    
def my_print(expr):
    global eviron_print_tex
    if eviron_print_tex:
        print('$$' + sym.latex(expr) + '$$')
    else:
        display(expr)
        
def batch_print(expr_list, punctuation='.', return_string=False):
    global eviron_print_tex
    if eviron_print_tex:
        print_str = '$$\\begin{align*}\n'
        for expr in expr_list:
            print_str += '\t' + sym.latex(expr).replace('=','&=') + '\\\\\n'
        # remove last new-line and add ending puncuation
        print_str = print_str[:-3] + punctuation + '\n'
        print_str += '\\end{align*}$$'
        if return_string:
            return print_str
        else:
            print(print_str)
    else:
        for expr in expr_list:
            display(expr)
            
def py_code(expr, replace_numpy=True, replace_unicode=True, numpy_package='np.'):
    code = sym.ccode(expr)
    
    if replace_unicode:
        unicode_replacements = [
            ('\\alpha', 'α'),
            ('\\beta', 'β'),
            ('\\gamma', 'γ'),
            ('\\Gamma', 'Γ'),
            ('\\delta', 'δ'),
            ('\\Delta', 'Δ'),
            ('\\epsilon', 'ϵ'),
            ('\\zeta', 'ζ'),
            ('\\eta', 'η'),
            ('\\theta', 'θ'),
            ('\\xi', 'ξ'),
            ('\\mu', 'μ')
        ]
        for unicode, symbol in unicode_replacements:
            code = code.replace(unicode, symbol)
    
    if replace_numpy:
        numpy_replacements = [
            ('exp', numpy_package + 'exp'),
            ('log', numpy_package + 'log'),
            ('pow', '( lambda base, exponent: base**exponent )'),
            ('Heaviside', '(lambda input: np.heaviside(input,0.5))')
        ]
        for c_func, np_func in numpy_replacements:
            code = code.replace(c_func, np_func)
        
    return code

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size': 24})

In [15]:
ξ, t, x, y = sym.symbols('\\xi t x y', real=True)
c, μ, α, γ, ϵ = sym.symbols('c \\mu \\alpha \\gamma \\epsilon', real=True, positive=True)
ν = sym.Function('\\nu')(t)
U = sym.Function('U')(ξ)
u1 = sym.Function('u1')(ξ, t)
u = sym.Function('u')(ξ, t)

w = sym.Function('w')
f = sym.Function('f')
F = sym.Integral(w(ξ,y)*f(u.subs(ξ,y)-a.subs(ξ,y)), (y, -sym.oo, sym.oo))
usub = U.subs(ξ, ξ-ϵ*ν) + ϵ*u1 + sym.O(ϵ**2)
a = sym.Function('a')(ξ, t)

In [17]:
eqn1 = c*μ*u.diff(ξ) - μ*u.diff(t) - F
eqn1

\mu*c*Derivative(u(\xi, t), \xi) - \mu*Derivative(u(\xi, t), t) - Integral(f(-a(y, t) + u(y, t))*w(\xi, y), (y, -oo, oo))

In [22]:
test = eqn1.subs(u, usub).doit().expand().collect(ϵ)
test

\epsilon*(\mu*c*Derivative(u1(\xi, t), \xi) + \mu*Derivative(\nu(t), t)*Subs(Derivative(U(_xi_1), _xi_1), _xi_1, -\epsilon*\nu(t) + \xi) - \mu*Derivative(u1(\xi, t), t)) + \mu*c*Subs(Derivative(U(_xi_1), _xi_1), _xi_1, -\epsilon*\nu(t) + \xi) - Integral(f(-a(y, t) + u(y, t))*w(\xi, y), (y, -oo, oo))

In [23]:
test.coeff(ϵ)

\mu*c*Derivative(u1(\xi, t), \xi) + \mu*Derivative(\nu(t), t)*Subs(Derivative(U(_xi_1), _xi_1), _xi_1, -\epsilon*\nu(t) + \xi) - \mu*Derivative(u1(\xi, t), t)